# TwelveCellDeconvoluteBloodEPIC

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)


Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.TwelveCellDeconvoluteBloodEPIC)


class TwelveCellDeconvoluteBloodEPIC(DeconvolutionSingleCell):
    def __init__(self):
        super().__init__()



In [3]:
model = pya.models.TwelveCellDeconvoluteBloodEPIC()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'twelvecelldeconvolutebloodepiccd4nv'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2024
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Ying, Kejun, et al. \"A unified framework for systematic curation and evaluation of aging biomarkers.\" Nature Aging (2025): 1-17."
model.metadata["doi"] = "https://doi.org/10.1038/s43587-025-00987-y"
model.metadata["research_only"] = None
model.metadata["notes"] = "Estimated proportion of CD4nv from 12-cell EPIC reference."


## Download clock dependencies

#### Download reference file

In [5]:
coeff_url = "https://raw.githubusercontent.com/bio-learn/biolearn/master/biolearn/data/twelve_cell_deconv.csv"
os.system(f"curl -L {coeff_url} -o twelve_cell_deconv.csv")


0

## Load features

#### From CSV file

In [6]:
import numpy as np
import pandas as pd

ref = pd.read_csv('twelve_cell_deconv.csv', index_col=0)
model.features = ref.index.astype(str).tolist()

ref_matrix = torch.tensor(ref.values, dtype=torch.float64)
pseudo_inv = torch.linalg.pinv(ref_matrix)

model.pseudo_inv = pseudo_inv
model.cell_index = 4
model.reference_values = torch.nanmean(ref_matrix, dim=1)


## Load weights into base model

#### From CSV file

In [7]:
# No linear base model; deconvolution logic in model forward


#### Linear model

In [8]:
model.base_model = None


## Load reference values

In [9]:
# reference_values already set above

## Load preprocess and postprocess objects

In [10]:
model.preprocess_name = "fill_with_reference_means"
model.preprocess_dependencies = None


In [11]:
model.postprocess_name = None
model.postprocess_dependencies = None


## Check all clock parameters

In [12]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Ying, Kejun, et al. "A unified framework for systematic curation '
             'and evaluation of aging biomarkers." Nature Aging (2025): 1-17.',
 'clock_name': 'twelvecelldeconvolutebloodepiccd4nv',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1038/s43587-025-00987-y',
 'notes': 'Estimated proportion of CD4nv from 12-cell EPIC reference.',
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2024}
reference_values: [0.9106685833333333, 0.8335768333333333, 0.8720161666666666, 0.8286470833333333, 0.92628525, 0.9155413333333334, 0.8792788333333332, 0.9127326666666665, 0.9067565833333332, 0.84800775, 0.10073516666666667, 0.14440299999999998, 0.049399916666666675, 0.08766050000000002, 0.07896441666666666, 0.12878258333333334, 0.16374524999999998, 0.13815541666666667, 0.17540216

## Basic Test

In [13]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[0.2505],
        [0.0000],
        [0.0000],
        [0.0213],
        [0.0000],
        [0.0000],
        [0.4266],
        [0.3712],
        [0.4565],
        [0.1010]], dtype=torch.float64)

## Save torch model


In [14]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")


## Clear directory
<a id="10"></a>


In [15]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: twelve_cell_deconv.csv
